In [8]:
import pandas as pd
from glob import glob
import os

In [6]:
list(glob('freshly_scraped/*.*'))

['freshly_scraped/ASTRO_Hubble_Source_Catalog_Search_API_Website.csv',
 'freshly_scraped/ASTRO_MAST_Documentation_Website.csv',
 'freshly_scraped/ASTRO_Calibration_Documentation_Website.csv',
 'freshly_scraped/ASTRO_NASA_Exoplanet_Archive_Documents_Website.csv',
 'freshly_scraped/ASTRO_HIRES_PRV_Website.csv',
 'freshly_scraped/ASTRO_Contributed_Datasets_Website.csv',
 'freshly_scraped/ASTRO_ZMAST_API_Website.csv',
 'freshly_scraped/ASTRO_Spitzer_Tools_Website.csv',
 'freshly_scraped/ASTRO_TAP_Search_Website.csv',
 'freshly_scraped/ASTRO_exoMAST_API_Website.csv',
 'freshly_scraped/ASTRO_Image_Cutouts_Website.csv',
 'freshly_scraped/ASTRO_API_Search_Website.csv',
 'freshly_scraped/ASTRO_Data_Reduction_Tools_Website.csv']

In [12]:
# see if we can match
for file in list(glob('freshly_scraped/*.*')):
    file_name = os.path.basename(file)
    file_name, ext = os.path.splitext(file_name)
    print(collection_ids[file_name])


1049
1047
1055
1059
1058
1057
1048
1051
1060
1046
1050
1061
1056


In [14]:
# load and convert to the format we want
for file in list(glob('freshly_scraped/*.*')):
    df = pd.read_csv(file)

In [16]:
import json

fresh_collections = glob('freshly_scraped/*.*')
print(len(fresh_collections))
def generate_json(row, config_folder):
    return {
        "model": "sde_collections.candidateurl",
        "fields": {
            "collection": collection_ids[config_folder],
            "url": row.url,
            "scraped_title": row.title,
            "generated_title": "",
            "level": 2,
            "visited": False
        }
    }

for collection in fresh_collections:
    file_name = os.path.basename(collection)
    config_folder, _ = os.path.splitext(file_name)

    with open(collection, "r") as f:
        directory = f'../scraper/scraped_urls/fresh/{config_folder}'

        if not config_folder in collection_ids:
            try:
                config_folder = replacement_dict[config_folder]
            except KeyError:
                print(f"Skipping {config_folder}")
                continue
        df = pd.read_csv(
            collection,
            names=["url", "title"],
        )
        df = df[["url", "title"]]
        if len(df) == 0:
            continue
        if not os.path.exists(directory):
            os.makedirs(directory)
        json_file_name = f"{directory}/urls.json"
        try:
            df = df.drop_duplicates(subset=['url'])
            # if config_folder == 'PDS_NASA_Science_Earths_Moon_Website':
            # remove lines that don't start with http
            df = df[df.url.str.lower().str.startswith('http')]
            df['json'] = df.apply(generate_json, args=(config_folder, ), axis=1)

            with open(json_file_name, "w") as f:
                f.write(json.dumps(df['json'].to_list(), indent=4))
        except KeyError:
            print(f"Some key error on: {config_folder}")

13


In [20]:
def generate_ingestion_commands():
    folders = glob('../scraper/scraped_urls/fresh/*')
    for folder in folders:
        folder = os.path.basename(os.path.normpath(folder))
        print(f"python manage.py loaddata scraper/scraped_urls/fresh/{folder}/urls.json")

generate_ingestion_commands()

python manage.py loaddata scraper/scraped_urls/fresh/ASTRO_Spitzer_Tools_Website/urls.json
python manage.py loaddata scraper/scraped_urls/fresh/ASTRO_Data_Reduction_Tools_Website/urls.json
python manage.py loaddata scraper/scraped_urls/fresh/ASTRO_ZMAST_API_Website/urls.json
python manage.py loaddata scraper/scraped_urls/fresh/ASTRO_Calibration_Documentation_Website/urls.json
python manage.py loaddata scraper/scraped_urls/fresh/ASTRO_exoMAST_API_Website/urls.json
python manage.py loaddata scraper/scraped_urls/fresh/ASTRO_Hubble_Source_Catalog_Search_API_Website/urls.json
python manage.py loaddata scraper/scraped_urls/fresh/ASTRO_Image_Cutouts_Website/urls.json
python manage.py loaddata scraper/scraped_urls/fresh/ASTRO_NASA_Exoplanet_Archive_Documents_Website/urls.json
python manage.py loaddata scraper/scraped_urls/fresh/ASTRO_HIRES_PRV_Website/urls.json
python manage.py loaddata scraper/scraped_urls/fresh/ASTRO_API_Search_Website/urls.json
python manage.py loaddata scraper/scraped_urls